In [9]:
import pandas as pd
import numpy as np
import operator

amazon_X = pd.read_csv("data/amazon_X.csv").to_numpy()
amazon_y = pd.read_csv("data/amazon_y.csv").to_numpy()
cancer_X = pd.read_csv("data/cancer_X.csv").to_numpy()
cancer_y = pd.read_csv("data/cancer_y.csv").to_numpy()
amazon_X.shape, amazon_y.shape, cancer_X.shape, cancer_y.shape

((1499, 10000), (1499, 1), (161, 217), (161, 1))

In [10]:
from sklearn.linear_model import LogisticRegression

reg = LogisticRegression()
reg.fit(amazon_X, amazon_y)
reg.score(amazon_X, amazon_y)

/Users/owemshu/opt/anaconda3/envs/deeplearning/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/owemshu/opt/anaconda3/envs/deeplearning/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


1.0

In [11]:
reg = LogisticRegression()
reg.fit(cancer_X, cancer_y)
reg.score(cancer_X, cancer_y)

/Users/owemshu/opt/anaconda3/envs/deeplearning/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/owemshu/opt/anaconda3/envs/deeplearning/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


1.0

In [12]:
class SoftmaxRegression:
    def __init__(self):
        self.n_samples = None
        self.n_features = None
        self.n_classes = None
        self.weights = None
        self.all_loss = list()

    def fit(self, x, y, iters=1000, alpha=0.1, lam=0.01):
        self.n_samples, self.n_features = x.shape
        self.n_classes = y.shape[1]
        # self.weights = np.random.rand(self.n_classes, self.n_features)  # 使用随机初始化会造成运算异常
        self.weights = np.zeros((self.n_classes, self.n_features))

        for i in range(iters):
            probs = self._softmax(x)
            loss = - (1 / self.n_samples) * np.sum(y * np.nan_to_num(np.log(probs)))
            self.all_loss.append(loss)

            dw = -(0.5 / self.n_samples) * np.dot((y - probs).T, x) + lam * self.weights
            dw[:, 0] -= lam * self.weights[:, 0]
            self.weights -= alpha * dw

    def predict(self, test_x):
        probs = self._softmax(test_x)
        return np.argmax(probs, axis=1).reshape((-1, 1))

    def score(self, test_x, test_y):
        score = 0.0
        for i in range(len(test_y)):
            if (self.predict([test_x[i]]) == np.argmax(test_y[i])).any():
                score += 1
        return score / len(test_y)

    def _softmax(self, x):
        scores = np.dot(x, self.weights.T)
        sm = np.exp(scores) / np.sum(np.exp(scores), axis=1, keepdims=True)
        sm = np.nan_to_num(sm)
        return sm

In [13]:
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
amazon_y = enc.fit_transform(amazon_y)

reg = SoftmaxRegression()
reg.fit(amazon_X, amazon_y, alpha=0.01)
reg.score(amazon_X, amazon_y)

/var/folders/dp/wdk87p0d3ml1ysm634x3wlf80000gn/T/ipykernel_18946/1685057548.py:17: RuntimeWarning: divide by zero encountered in log
  loss = - (1 / self.n_samples) * np.sum(y * np.nan_to_num(np.log(probs)))
/Users/owemshu/opt/anaconda3/envs/deeplearning/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


1.0

In [14]:
np.random.seed(64)
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
cancer_y = enc.fit_transform(cancer_y)

reg = SoftmaxRegression()
reg.fit(cancer_X, cancer_y)
reg.score(cancer_X, cancer_y)

1.0

In [ ]:
from sklearn.preprocessing import Normalizer
from sklearn.decomposition import PCA

pca = PCA(n_components=200)
amazon_X = pca.fit_transform(amazon_X)
print(sum(pca.explained_variance_ratio_))

normalizer = Normalizer()
amazon_X = normalizer.fit_transform(amazon_X)

In [ ]:
def square_distance(v1, v2):
    sum = 0
    for i in range(len(v1)):
        if v1[i] != v2[i]:
            sum += 1
    return sum


def vote(ys):
    vote_dict = {}
    for y in ys:
        y = y[0]
        if y not in vote_dict.keys():
            vote_dict[y] = 1
        else:
            vote_dict[y] += 1
    sorted_vote_dict = sorted(vote_dict.items(), key=operator.itemgetter(1), reverse=True)
    return sorted_vote_dict[0][0]  # 将已经记好的数进行一个自大而小的排列(则是一个降序排序），若reverse是false,则是自小而大.(升序排序）


class KNN:
    def __init__(self, k=3):
        self.x = None
        self.y = None
        self.k = k

    def fit(self, x, y):
        self.x = x
        self.y = y

    def predict(self, x):
        y_pred = []
        for i in range(len(x)):
            dist_arr = [square_distance(x[i], self.x[j]) for j in range(len(self.x))]
            sorted_index = np.argsort(dist_arr)
            top_k_index = sorted_index[:self.k]
            y_pred.append(vote(ys=self.y[top_k_index]))  # 调用了vote函数，去进行分类
        return np.array(y_pred)

    def score(self, y_true=None, y_pred=None):
        if y_true is None and y_pred is None:
            y_pred = self.predict(self.x)
            y_true = self.y
        score = 0.0
        for i in range(len(y_true)):
            if y_true[i] == y_pred[i]:
                score += 1
        return score / len(y_true)

In [ ]:
amazon_y = pd.read_csv("data/amazon_y.csv").to_numpy()
cancer_y = pd.read_csv("data/cancer_y.csv").to_numpy()

In [ ]:
clf = KNN()
clf.fit(amazon_X, amazon_y)
print(clf.score())

In [ ]:
clf = KNN()
clf.fit(cancer_X, cancer_y)
print(clf.score())